***GENERATED CODE FOR stdscalartestclustering PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'sex', 'transformation_label': 'String Indexer'}], 'feature': 'sex', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '80', 'mean': '', 'stddev': '', 'min': 'female', 'max': 'male', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'sex'}, {'feature_label': 'sex', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('sex')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'age', 'transformation_label': 'String Indexer'}], 'feature': 'age', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '80', 'mean': '', 'stddev': '', 'min': '0s', 'max': '80s', 'missing': '0', 'distinct': '8'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'age'}, {'feature_label': 'age', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('age')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'country', 'transformation_label': 'String Indexer'}], 'feature': 'country', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '80', 'mean': '', 'stddev': '', 'min': 'Korea', 'max': 'Korea', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'country'}, {'feature_label': 'country', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('country')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'province', 'transformation_label': 'String Indexer'}], 'feature': 'province', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '80', 'mean': '', 'stddev': '', 'min': 'Seoul', 'max': 'Seoul', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'province'}, {'feature_label': 'province', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('province')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'city', 'transformation_label': 'String Indexer'}], 'feature': 'city', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '80', 'mean': '', 'stddev': '', 'min': 'Dongdaemun-gu', 'max': 'Yongsan-gu', 'missing': '0', 'distinct': '21'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'city'}, {'feature_label': 'city', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('city')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'infection_case', 'transformation_label': 'String Indexer'}], 'feature': 'infection_case', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '80', 'mean': '', 'stddev': '', 'min': 'Dongan Church', 'max': 'overseas inflow', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'infection_case'}, {'feature_label': 'infection_case', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('infection_case')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'state', 'transformation_label': 'String Indexer'}], 'feature': 'state', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '80', 'mean': '', 'stddev': '', 'min': 'released', 'max': 'released', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'state'}, {'feature_label': 'state', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('state')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
import json
from sklearn.cluster import KMeans
import optuna
import numpy as np
from sklearn import metrics
import pandas as pd


def kmeans(df, TrainingPercent):
    rows = df.count()
    percentrow = TrainingPercent / 100 * rows
    percentrows = int(percentrow)
    df = df.limit(percentrows)
    pd_df = df.toPandas()

    def objective(trial):
        n_clusters = int(trial.suggest_loguniform("n_clusters", 2, 10))
        classifier_obj = KMeans(n_clusters=n_clusters).fit(pd_df)
        labels = classifier_obj.predict(pd_df)
        score = metrics.silhouette_score(pd_df, labels, metric='euclidean')
        return score
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)
    best = study.best_params
    best_param = int(best["n_clusters"])
    model = KMeans(n_clusters=best_param).fit(pd_df)
    labels = model.predict(pd_df)
    silhouette = metrics.silhouette_score(pd_df, labels, metric='euclidean')
    pd_df['prediction'] = pd.DataFrame(labels)
    labels_unique = np.unique(labels)
    totalClusters = len(labels_unique)
    display("totalClusters     : %s" % totalClusters)
    display(pd_df.head())
    return pd_df


class Clustering:

    def run(spark_DF, spark, config):
        stageAttributes = json.loads(config)
        stageAttributes['model']
        trainingPercent = stageAttributes['TrainingPercent']
        autokmeans_obj = kmeans(df=spark_DF, TrainingPercent=trainingPercent)
        [autokmeans_obj]


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run stdscalartestclusteringHooks.ipynb
try:
	#sourcePreExecutionHook()

	coronapatientjeuk = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/testdata/1711602597939_Corona_Patient.csv', 'filename': 'Corona_Patient.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'FilePath': '/SampleSolutions/Corona/Corona_Patient.csv', 'viewFileName': 'Corona_Patient.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://dnm.bfirst.ai:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run stdscalartestclusteringHooks.ipynb
try:
	#transformationPreExecutionHook()

	stdscalartestclusteringautofe = TransformationMain.run(coronapatientjeuk,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "sex", "transformation_label": "String Indexer"}], "feature": "sex", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "80", "mean": "", "stddev": "", "min": "female", "max": "male", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "sex"}, {"transformationsData": [{"feature_label": "age", "transformation_label": "String Indexer"}], "feature": "age", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "80", "mean": "", "stddev": "", "min": "0s", "max": "80s", "missing": "0", "distinct": "8"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "age"}, {"transformationsData": [{"feature_label": "country", "transformation_label": "String Indexer"}], "feature": "country", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "80", "mean": "", "stddev": "", "min": "Korea", "max": "Korea", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "country"}, {"transformationsData": [{"feature_label": "province", "transformation_label": "String Indexer"}], "feature": "province", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "80", "mean": "", "stddev": "", "min": "Seoul", "max": "Seoul", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "province"}, {"transformationsData": [{"feature_label": "city", "transformation_label": "String Indexer"}], "feature": "city", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "80", "mean": "", "stddev": "", "min": "Dongdaemun-gu", "max": "Yongsan-gu", "missing": "0", "distinct": "21"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "city"}, {"transformationsData": [{"feature_label": "infection_case", "transformation_label": "String Indexer"}], "feature": "infection_case", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "80", "mean": "", "stddev": "", "min": "Dongan Church", "max": "overseas inflow", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "infection_case"}, {"transformationsData": [{"feature_label": "state", "transformation_label": "String Indexer"}], "feature": "state", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "80", "mean": "", "stddev": "", "min": "released", "max": "released", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "state"}]}))

	#transformationPostExecutionHook(stdscalartestclusteringautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run stdscalartestclusteringHooks.ipynb
try:
	#mlPreExecutionHook()

	stdscalartestclusteringclustering = Clustering.run(stdscalartestclusteringautofe,spark,json.dumps( {"autoClustering": 1, "originalfile": "/FileStore/platform/testdata/1711602597939_Corona_Patient.csv", "model": "Kmeans", "TrainingPercent": 100, "totalClusters": 2}))

	#mlPostExecutionHook(stdscalartestclusteringclustering)

except Exception as ex: 
	logging.error(ex)
#spark.stop()
